In [ ]:
import os, glob
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFilter

In [ ]:
# Place 128*128 pixel cut tiff images under the file_directory that you define below
file_directory = '/XXX/XXX/XXX/XXX/XXX'
img_list = sorted(glob.glob(file_directory + '/*.tif'))

In [ ]:
# The following function was adopted from 
# https://note.nkmk.me/python-pillow-square-circle-thumbnail/

def mask_circle_solid(pil_img, background_color, blur_radius, offset=0):
        background = Image.new(pil_img.mode, pil_img.size, background_color)
        offset = blur_radius * 2 + offset
        mask = Image.new("L", pil_img.size, 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((offset, offset, pil_img.size[0] - offset, pil_img.size[1] - offset), fill=255)
        mask = mask.filter(ImageFilter.GaussianBlur(blur_radius))
        return Image.composite(pil_img, background, mask)
    
# The following functions were adopted from 
# https://teratail.com/questions/157585

def complex_plane(width):
    half_width = width // 2
    re = np.array(range(width)) - half_width
    im = - re
    re, im = np.meshgrid(re, im)
    return re + im * 1j
    
def aggregate_in_angle(agg_fun, spectrum, min_angle, max_angle):
    width = spectrum.shape[0]
    assert spectrum.shape[1] == width
    min_radius, max_radius = 0, width // 2
    assert 0 <= min_angle < max_angle <= 180  # 単位はdegree
    cp = complex_plane(width)
    cp_mag = np.abs(cp)
    in_radius = np.logical_and(min_radius < cp_mag, cp_mag < max_radius)
    cp_angle = np.angle(cp, deg=True)
    in_angle = np.logical_and(min_angle <= cp_angle, cp_angle < max_angle)
    return agg_fun(spectrum[np.logical_and(in_radius, in_angle)])
    
def test(spectrum, angle_gap):
    means = [aggregate_in_angle(np.mean, spectrum, angle, angle + angle_gap) for angle in range(0, 180, angle_gap)]
    return means

In [ ]:
Img_list = []
MagMin_list = []
MagMax_list = []
DamageIndex_list = []

for i in img_list:
    im_circ = mask_circle_solid(Image.open(i), (0, 0, 0), 2).convert("L")
    f_im_circ = np.fft.fft2(im_circ)
    f_im_circ = np.fft.fftshift(f_im_circ)
    mag = 20*np.log(np.abs(f_im_circ))
    MagMin = np.min(test(mag,45))
    MagMax = np.max(test(mag,45))
    DamageIndex = MagMin/MagMax
    Img_list.append(os.path.basename(i))
    MagMin_list.append(MagMin)
    MagMax_list.append(MagMax)
    DamageIndex_list.append(DamageIndex)

In [ ]:
df = pd.DataFrame([Img_list, MagMin_list, MagMax_list, DamageIndex_list]).T
df.to_csv("DamageIndex.csv", header=False, index=False)